<a href="https://colab.research.google.com/github/hsanthan/qiskit-machine-learning/blob/main/qsvc_voice_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install qiskit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 2.3 MB/s 
     |████████████████████████████████| 18.0 MB 348 kB/s 
     |████████████████████████████████| 240 kB 54.1 MB/s 
     |████████████████████████████████| 198 kB 57.5 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 112 kB 54.6 MB/s 
     |████████████████████████████████| 1.6 MB 37.1 MB/s 
     |████████████████████████████████| 38.1 MB 48.0 MB/s 
     |████████████████████████████████| 37.5 MB 1.3 MB/s 
     |████████████████████████████████| 49 kB 6.6 MB/s 
     |████████████████████████████████| 49 kB 6.3 MB/s 
     |████████████████████████████████| 943 kB 36.1 MB/s 
     |████████████████████████████████| 4.0 MB 38.5 MB/s 
     |████████████████████████████████| 112 kB 54.4 MB/s 
  Created wheel for qiskit: filename=qiskit-0.36.2-py3-none-any.whl size=11933 sha256=8825f4df1

In [24]:
!pip install qiskit_machine_learning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 4.3 MB/s 


In [4]:
import qiskit

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [6]:
voice = pd.read_csv('voice.csv')

In [7]:
voice.head()

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,...,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,...,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,...,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male
2,0.077316,0.083829,0.036718,0.008701,0.131908,0.123207,30.757155,1024.927705,0.846389,0.478905,...,0.077316,0.098706,0.015656,0.271186,0.007990,0.007812,0.015625,0.007812,0.046512,male
3,0.151228,0.072111,0.158011,0.096582,0.207955,0.111374,1.232831,4.177296,0.963322,0.727232,...,0.151228,0.088965,0.017798,0.250000,0.201497,0.007812,0.562500,0.554688,0.247119,male
4,0.135120,0.079146,0.124656,0.078720,0.206045,0.127325,1.101174,4.333713,0.971955,0.783568,...,0.135120,0.106398,0.016931,0.266667,0.712812,0.007812,5.484375,5.476562,0.208274,male


In [8]:
#Shuffle
voice = voice.sample(frac=1).reset_index(drop=True)

In [9]:
X = voice.drop('label',axis=1)
y = voice['label']

In [12]:
from sklearn import preprocessing
labels = preprocessing.LabelEncoder()
labels.fit(y)
labels = labels.transform(y)

In [13]:
labels

array([1, 0, 1, ..., 1, 1, 0])

In [17]:
from sklearn.preprocessing import MinMaxScaler

features = MinMaxScaler().fit_transform(X)

In [20]:
features

array([[0.458039  , 0.6511354 , 0.43594035, ..., 0.18227305, 0.18240343,
        0.07976903],
       [0.81660292, 0.18387975, 0.79452997, ..., 0.37889787, 0.37854077,
        0.13227083],
       [0.66551225, 0.43029501, 0.70403987, ..., 0.45854182, 0.45815451,
        0.12264672],
       ...,
       [0.73025992, 0.43324214, 0.72590876, ..., 0.36418871, 0.35622318,
        0.12709138],
       [0.69596416, 0.38788975, 0.75910756, ..., 0.07576841, 0.0751073 ,
        0.09916654],
       [0.75634528, 0.4495499 , 0.79744847, ..., 0.0189421 , 0.01287554,
        0.14992367]])

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.30)

In [19]:
X_train.shape

(2217, 20)

In [21]:
X_test.shape

(951, 20)

In [22]:
# number of qubits is equal to the number of features
num_qubits = 20

# number of steps performed during the training procedure
tau = 100

# regularization parameter
C = 1000


In [25]:
from qiskit import BasicAer
from qiskit.circuit.library import ZFeatureMap
from qiskit.utils import QuantumInstance, algorithm_globals

from qiskit_machine_learning.kernels import  QuantumKernel

algorithm_globals.random_seed = 12345

pegasos_backend = QuantumInstance(
    BasicAer.get_backend("statevector_simulator"),
    seed_simulator=algorithm_globals.random_seed,
    seed_transpiler=algorithm_globals.random_seed,
)

feature_map = ZFeatureMap(feature_dimension=num_qubits, reps=1)

qkernel = QuantumKernel(feature_map=feature_map, quantum_instance=pegasos_backend)

In [26]:
from qiskit_machine_learning.algorithms import PegasosQSVC

pegasos_qsvc = PegasosQSVC(quantum_kernel=qkernel, C=C, num_steps=tau)

# training
pegasos_qsvc.fit(X_train, y_train)

# testing
pegasos_score = pegasos_qsvc.score(X_test, y_test)
print(f"PegasosQSVC classification test score: {pegasos_score}")

PegasosQSVC classification test score: 0.8727655099894848


In [28]:
predictions = pegasos_qsvc.predict(X_test)

In [29]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,predictions))

[[389 104]
 [ 17 441]]


In [30]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.96      0.79      0.87       493
           1       0.81      0.96      0.88       458

    accuracy                           0.87       951
   macro avg       0.88      0.88      0.87       951
weighted avg       0.89      0.87      0.87       951

